In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.opts.defaults(hv.opts.Curve(width=500), 
                 hv.opts.Histogram(width=500))

In [ ]:
import numpy as np

# Gaussian Mixture Models


## Introduction

**Unsupervised methods:** Contrary to supervised (regression, classification) in this scenario there are no labels/targets. In general the objective of unsupervised methods is **to find structure in the data**. We do this to better understand the data: find hidden causes, find shared features, etc

This tasks can be solve using **Latent variable models (LVM)**. An LVM models a **cause and effect** structure 

- Observed variables (effect): Data
- Latent variables (cause): Hidden variables that explains the observed variables 

and the objective is to infer the latent given the observed. In general we have to assume properties for the latent variable, for example

- Dimension of the latent is smaller than the observed (Dimensionality reduction)
- Latent variable is continuous and Gaussian distributed (PCA)
- Latent variables are not correlated (observed data might be)

In this lesson we will focus on LVM models with discrete latent variables for clustering

> Clustering: Group the data into clusters (sets) such that
> - "Similar" data samples go to the same cluster. We have to define similarity
> - Elements of one cluster are "different" from the elements of another cluster







## Mixture model

A mixture model is an LVM for clustering. We represent each data point 

$$
x_i \in \mathbb{R}^D \text{ with } i=1,\ldots, N
$$

as having a **categorical latent variable**

$$
z_i \in {1, 2, \ldots, K}
$$

note that this is the same as saying that $z_i$ has a categorical prior

$$
p(z_i) = \text{Cat}(\vec \pi),
$$

where $\pi_k \in [0, 1]$ are the mixing coefficients and $\sum_{k=1}^K \pi_k=1$ 

:::{important}

We assume that there are $K$ clusters (sets). The latent variable associates each data point with one of the $K$ clusters

:::


We can write the likelihood of the mixture model as

$$
\begin{align}
p(x_i | \theta) &= \sum_{k=1}^K p(x_i|z_i = k, \theta) p(z_i=k| \theta) \nonumber \\
&= \sum_{k=1}^K p_k(x_i| \theta_k) \pi_k ,  \nonumber
\end{align}
$$

To continue we have to specify the family of $p_k(x_i|\theta_k)$

## Gaussian mixture model (GMM)

In the GMM we further assume that each component of the mixture is a Gaussian/Normal distribution. With this we can write the likelihood as 

$$
p(x_i | \theta) = \sum_{k=1}^K \pi_k \mathcal{N}(x_i|\mu_k, \Sigma_k),
$$

where each cluster has three parameters

- The mean of the clusters $\mu_k$, which represents the location of the center of the clusters
- The covariance $\Sigma_k$, which gives the size and orientation of the clusters
- The concentration of the cluster $\pi_k$

The fitting the GMM is: **Finding the best value of these parameters** for all clusters given the data

Note that the GMM is a **generative model**, given a value for the parameters we can sample from the GMM

Let's explore the influence of these parameters on the "shape" of the mixture

In [ ]:
def create_mixture(pis, mus, Sigmas, N=5000, rseed=None):
    """
    pis (list of floats): mixing coefficients : cluster importance
    mus (list of arrays): mean: cluster centers
    Sigmas (list of arrays): covariance: cluster shape
    N (int): number of points to create
    """
    if rseed is not None:
        np.random.seed(rseed)
    points = list()
    labels = list()
    for k in range(len(pis)):
        Nc = int(pis[k]*N)
        if len(mus[k]) > 1:
            x = np.random.multivariate_normal(mus[k], Sigmas[k], Nc)
        else:
            x = mus[k] + np.sqrt(Sigmas[k])*np.random.randn(Nc)
        points.append(x)
        labels.append([k]*Nc)
    return np.concatenate(points), np.concatenate(labels)

In [ ]:
data, labels = create_mixture(pis=[0.4, 0.5, 0.1], 
                              mus=[[-1], [2], [2.5]], 
                              Sigmas=[0.1, 0.5, 0.1])

edges, bins = np.histogram(data, bins=50, density=True)

In [ ]:
hv.Histogram((edges, bins), kdims='data', vdims='Density')

In [ ]:
data, labels = create_mixture(pis=[0.9, 0.1], 
                              mus=[[-1, -1], [1, 1]], 
                              Sigmas=[0.5*np.eye(2), [[1, -0.9],[-0.9, 1]]])

bins, edgesx, edgesy = np.histogram2d(data[:, 0], data[:, 1], bins=30, density=True)

In [ ]:
hv.Scatter((data[:, 0], data[:, 1])) +  hv.Image((edgesx, edgesy, bins))

### Posterior probability a.k.a. picking a cluster

Let's assume that we know the parameters of the mixture and we want to infer the most probable cluster for a given data point

> This is given by the posterior probability of $z$ given $x$

Using the Bayes rule and our previous assumptions (likelihood) we have

$$
\begin{align}
r_{ik} = p(z_i = k|x_i, \theta) &= \frac{ p(x_i|z_i = k, \theta) p(z_i=k| \theta)}{p(x_i)} \nonumber \\
&= \frac{ p(x_i|z_i = k, \theta) p(z_i=k| \theta)}{ \sum_{k=1}^K p(x_i|z_i = k, \theta) p(z_i=k| \theta) } \nonumber \\
&= \frac{ \mathcal{N}(x_i|\mu_k, \Sigma_k)  \pi_k}{ \sum_{k=1}^K \mathcal{N}(x_i|\mu_k, \Sigma_k) \pi_k }, \nonumber\end{align}
$$

this posterior is also called the **responsibility** $r_{ik}$ of cluster $k$ for point $i$

:::{note}

It is a **soft-cluster assignment**

:::

In some cases we only need a **hard-cluster assignment**, *i.e.* the single most probable cluster, which can be obtained as

$$
c_{i} = \text{arg}\max_k r_{ik} = \text{arg}\max_k \log \mathcal{N}(x_i|\mu_k, \Sigma_k) + \log \pi_k,
$$

because of the maximum operator we can omit the evidence (denominator)

In [ ]:
def jointGMM(data, pi, mu, Sigma):
    # Assume 2 dimensional data
    data_centered = data - mu
    norm = pi/(2*np.pi*np.sqrt(np.linalg.det(Sigma)))
    xSx = np.multiply(data_centered, np.linalg.solve(Sigma, data_centered.T).T)
    return norm*np.exp(-0.5*np.sum(xSx, axis=1))

pis = [0.6, 0.4]; mus=[[-1, -1], [1, 1]]; 
Sigmas=[0.5*np.eye(2), [[1, -0.3],[-0.3, 1]]]
data, labels = create_mixture(pis, mus, Sigmas, rseed=0)

posterior = np.zeros(shape=(len(data), len(pis)))
for k in range(len(pis)):
    posterior[:, k] = jointGMM(data, pis[k], mus[k], Sigmas[k])
posterior = posterior/np.sum(posterior, axis=1)[:, np.newaxis]

In [ ]:
p1 = hv.Scatter((data[:, 0], data[:, 1], posterior[:, 1]), 
                vdims=['y', 'z'], label='Soft assignment').opts(color='z')
p2 = hv.Scatter((data[:, 0], data[:, 1], np.argmax(posterior, axis=1)), 
                vdims=['y', 'z'], label='Hard assignment').opts(color='z')
hv.Layout([p1, p2]).cols(2).opts(hv.opts.Scatter(cmap='RdBu'))

### Fitting the GMM

We want to find the optimal point estimates of the parameters of a statistical model. Can we use **MLE**? The log likelihood in this case is 

$$
\log L(\pi,\mu,\Sigma) = \sum_{i=1}^N \log \sum_{k=1}^K \pi_k \mathcal{N}(x_i|\mu_k, \Sigma_k)
$$

:::{warning}

With the exception of $K=1$, we won't be able to find analytic expressions for the parameters

:::

In general when we don't have complete data, i.e. there are missing or hidden variables, the posterior does not factorize

In this case we have the latent/hidden variables $z_i$. We need to marginalize it (sum over k) to compute the likelihood. In what follows we will review approximate MLE estimation: **Expectation Maximization (EM)**

Other problems of the MLE/MAP for GMM

- Non-convex, it has local optima (Murphy 11.3.2)
- Very hard to compute (NP-complete hard!)
- **Unidentifiability:** There exist $K!$ configurations that yield the exact same solution (label switch)
- **Singularities:** A component may collapse to a single point ($\Sigma \to 0$ and $\log L \to \infty$)

## Expectation Maximization (EM)

The MLE solution can be computed if we **"fully-observe"** the data. This means that

> We know $z_i$ for each $x_i$, *i.e.* we know which cluster owns $x_i$

The **fully-observed** log likelihood (FOLL) is

$$
\log L_{\text{FO}}(\theta) = \sum_{i=1}^N \log p(z_i) p(x_i|z_i, \theta), 
$$

but in practice we do not observe $z$, so we marginalize it by computing the **expected value** of the FOLL given the posterior

$$
\begin{align}
Q(\theta, \theta^{\text{old}}) &= \mathbb{E}_{p(z|x, \theta^{\text{old}})} \left[\log L_{\text{FO}}(\theta) \right]   \nonumber \\
&= \sum_{k=1}^K \sum_{i=1}^N p(k|x_i, \theta^{\text{old}})  \log p(k) p(x_i|\theta_k) \nonumber
\end{align}
$$

which is called the **auxiliary function**. After this we update our parameters by **maximizing** 

$$
\theta^{\text{new}} = \text{arg}\max_\theta Q(\theta, \theta^{\text{old}}),
$$

and finally we do $\theta^{\text{old}} \leftarrow \theta^{\text{new}}$ and repeat the procedure until convergence


This iterative procedure of

1. **E-step:** Estimating the expected value of the likelihood given our current parameters
1. **M-step:** Maximizing the estimator and updating our parameters


is called **Expectation Maximization** (EM)


EM is a general algorithm:

- it can be used for other problems with non-analytical solution/non-totally observed data
- it has many variants: Incremental, Variational, Monte-Carlo (Murphy 11.4.8 and 11.4.9)

### EM updates for the GMM

Let's start by defining the auxiliary function for the GMM

$$
\begin{align}
Q(\theta, \theta^{\text{old}}) 
&= \sum_{k=1}^K \sum_{i=1}^N p(k|x_i, \theta^{\text{old}})  \log p(k) p(x_i|\theta_k)  \nonumber \\
&= \sum_{k=1}^K \sum_{i=1}^N r_{ik}^{\text{old}}  \left(\log \pi_k + \log \mathcal{N}(x_i|\mu_k, \Sigma_k) \right),\nonumber
\end{align}
$$

where $r_{ik}^{\text{old}}$ is the responsibility given the parameters of the previous iteration. 

Now we maximize $Q$ to get an estimator of the parameters

The estimator for the means is given by

$$
\mu_{k}^{\text{new}}  = \frac{1}{\sum_{i=1}^N r_{ik}^{\text{old}}} \sum_{i=1}^N r_{ik}^{\text{old}} x_i
$$

Which is obtained by setting the derivative of $Q$ to zero


:::{dropdown} Proof

$$
\begin{align}
\frac{d Q}{d\mu_{k'}} &= \frac{d}{d\mu_{k'}} \sum_{k=1}^K \sum_{i=1}^N r_{ik}^{\text{old}}  \left(\log \pi_k + \log \mathcal{N}(x_i|\mu_k, \Sigma_k) \right) \nonumber \\
&= \sum_{i=1}^N r_{ik'}^{\text{old}}  \frac{d}{d\mu_{k'}}  \log \mathcal{N}(x_i|\mu_{k'}, \Sigma_{k'})\nonumber \\
&= \sum_{i=1}^N r_{ik'}^{\text{old}}  \frac{d}{d\mu_{k'}} \left( -\frac{1}{2} (x_i - \mu_{k'})^T \Sigma_{k'}^{-1} (x_i - \mu_{k'}) -\frac{D}{2} \log(2\pi) -\frac{1}{2} \log(|\Sigma_{k'}|) \right)\nonumber \\
&= -\frac{1}{2} \sum_{i=1}^N r_{ik'}^{\text{old}}  \frac{d}{d\mu_{k'}} (x_i - \mu_{k'})^T \Sigma_{k'}^{-1} (x_i - \mu_{k'})  \nonumber \\
&= \sum_{i=1}^N r_{ik'}^{\text{old}} \Sigma_{k'}^{-1} (x_i - \mu_{k'})  = 0 \nonumber
\end{align}
$$

:::




Following the same procedure for the covariance we arrive to

$$
\begin{align}
\Sigma_k^{\text{new}} &= \frac{1}{\sum_{i=1}^N r_{ik}^{\text{old}}} \sum_{i=1}^N r_{ik}^{\text{old}} (x_i - \mu_k) (x_i - \mu_k)^T \nonumber \\
&= -\mu_k \mu_k^T  + \frac{1}{\sum_{i=1}^N r_{ik}^{\text{old}}} \sum_{i=1}^N r_{ik}^{\text{old}} x_i x_i^T   \nonumber
\end{align}
$$

Finally, by incorporating the constraint $\sum_k \pi_k = 1$ we arrive to 

$$
\pi_k^{\text{new}} = \frac{\sum_{i=1}^N r_{ik}^{\text{old}}}{N}
$$


### Summary

The following algorithms summarizes the application of EM to the GMM problem

- **(1)** Set initial conditions
- **(2)** Compute cluster responsibilities for each data point using

$$
r_{ik}^{\text{old}} = \frac{ \mathcal{N}(x_i|\mu_k^{\text{old}}, \Sigma_k^{\text{old}})  \pi_k^{\text{old}}}{ \sum_{k=1}^K \mathcal{N}(x_i|\mu_k^{\text{old}}, \Sigma_k^{\text{old}}) \pi_k^{\text{old}} }
$$

- **(3)** Compute the new values for the parameters using

$$
\mu_{k}^{\text{new}}  = \frac{1}{\sum_{i=1}^N r_{ik}^{\text{old}}} \sum_{i=1}^N r_{ik}^{\text{old}} x_i,
$$

$$
\Sigma_k^{\text{new}} = -\mu_k \mu_k^T  + \frac{1}{\sum_{i=1}^N r_{ik}^{\text{old}}} \sum_{i=1}^N r_{ik}^{\text{old}} x_i x_i^T , 
$$

and

$$
\pi_k^{\text{new}} = \frac{\sum_{i=1}^N r_{ik}^{\text{old}}}{N}
$$

- **(4)** Update the parameters $\theta^{\text{old}} \leftarrow \theta^{\text{new}}$

- **(5)** If convergence criterion is met stop otherwise go back to **2**

### Example using "Old faithful" data

Iteration by iteration  (Bishop Fig. 9.8)

<img src="img/gmm_bishop.png" width="700">

Animation (from wikipedia)

<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/EM_Clustering_of_Old_Faithful_data.gif" width="300">

## Practical details of EM

**Initialization and restarts** 

The EM procedure is guaranteed to converge to a *stationary point* (local minimum/maximum or saddle point)

- One can obtain a better solution using the following heuristic:
    1. Set a **number of retries**
    1. Run GMM with a random initialization
    1. If the likelihood of this solution is larger than the previous replace best
    1. Repeat until **number of retries** is achieved
- Another option is to initialize with the solution from a simpler algorithms (*e.g.* k-means)
    - This can avoid catastrophic results
    - But it can also limit exploration 
    
**Convergence**

EM is an iterative procedure

> How many times do we repeat it?

- Check log likelihood vs iteration
- For example: stop if log likelihood increment is less than 0.1\% wrt to previous

**Number of clusters**

The number of clusters $K$ is a parameter set *a priori* by the user

> How do we know what value of $K$ is good?

Option 1: The Elbow method: Plot log likelihood (in the validation set) as a function of $K$

The log likelihood may keep decreasing with $K$ (overfit) but at some point the decrease is negligible: find the **elbow** of the curve

<img src="img/elbow.png" width="500">

Option 2: Use the Akaike Information Criterion (AIC) or the Bayesian Information Criterion (BIC), or other criterion for model selection that penalizes complexity

Option 3: [The Silhouette score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)


## Gaussian mixture with [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)

From the `sklearn.mixture` scikit learn module

```python
GaussianMixture(n_components=1, # Number of clusters
                covariance_type='full', # Shape of the covariance
                tol=0.001, # termination tolerance of EM
                reg_covar=1e-06, # small number added to the diagonal of the covariance
                max_iter=100, # Max number of EM iterations
                n_init=1, # Number of random initializations
                init_params='kmeans', # Initialization for the responsabilities (kmeans or random)
                ...
               )
```

The shape can be `full`, `tied`, `diag` or `spherical`. 

- In a diagonal covariance $\Sigma = \vec \sigma^2 \text{I}$, $\vec \sigma \in \mathbb{R}_{+}^D$, clusters cannot have rotations but the variance of each dimension can be different
- In a spherical covariance $\Sigma = \sigma^2 \text{I}$, $\sigma \in \mathbb{R}_{+}$, clusters cannot have rotations and all dimensions have the same variance
- Tied covariance means that all clusters have the same covariance (dispersion and rotation)




Let's test this module. We start by creating synthetic data as before

In [ ]:
data, labels = create_mixture(pis=[0.33, 0.33, 0.34], 
                              mus=[[-3, 0], [0, 0], [3 ,0]], 
                              Sigmas=[[[1, 0.9],[0.9, 1]], 
                                      [[1, -0.9],[-0.9, 1]], 
                                      [[1, 0.9],[0.9, 1]]])

In [ ]:
hv.Scatter((data[:, 0], data[:, 1])).opts(width=500)

We will use the Bayesian information criterion (BIC) to find the "best" number of clusters. 

$$
BIC = K \log N - 2 \log L
$$

The BIC grows with $k$ the number of parameters and number of samples $n$ and decreases with the maximum value of the log likelihood. A model with lowest BIC is preferred. 

In [ ]:
from sklearn.mixture import GaussianMixture

maxK = 20; 
cov = 'full' 
metrics = np.zeros(shape=(maxK-1, ))
for i, K in enumerate(range(2, maxK+1)):
    gmm = GaussianMixture(n_components=K, covariance_type=cov, n_init=3)
    gmm.fit(data)
    metrics[i] = gmm.bic(data)

In [ ]:
hv.Curve((range(2, maxK+1), metrics), 'K', 'BIC')

The minimum BIC is obtained using three clusters. Let's use $K=3$ and inspect the results

In [ ]:
gmm = GaussianMixture(n_components=3, covariance_type=cov, n_init=3)
gmm.fit(data)
hatr = gmm.predict(data)

The hard assignments:

In [ ]:
hv.Scatter((data[:, 0], data[:, 1], hatr), vdims=['y', 'z'],).opts(width=500, color='z', cmap='Category10')

The entropy (uncertainty) of the soft assignments (responsabilities):

In [ ]:
p = gmm.predict_proba(data)
H = -np.sum(p*np.log(p), axis=1)

In [ ]:
hv.Scatter((data[:, 0], data[:, 1], H), vdims=['y', 'z'],).opts(width=500, color='z', cmap='Blues', colorbar=True)

The generative results:

In [ ]:
hatdata = gmm.sample(n_samples=10000)
hatdata

In [ ]:
hv.Scatter((hatdata[0][:, 0], hatdata[0][:, 1])).opts(width=500)

## Appendix: Relation to K-means

The K-means algorithm is a GMM with the following constraints

- Spherical clusters: $\Sigma = \sigma^2 \text{I}$, $\sigma \in \mathbb{R}_{+}$
- All clusters have equal size: $\sigma^2 = \sigma_1^2 = \sigma_2^2 = \ldots = \sigma_K^2$
- Uniform prior for the mixture coefficients: $\pi_k = \frac{1}{K}$
- Hard labels are used instead of responsibilities

The multivariate normal for spherical clusters

$$
\mathcal{N}(x_i|\mu_k, \Sigma_k = \sigma^2 I) = \frac{1}{\sqrt{(2\pi)^D} \sigma} \exp \left(-\frac{1}{2\sigma^2} \|x_i - \mu_k\|^2 \right)
$$

The update rule for the responsibility 

$$
\begin{align}
\hat r_{ik}^{\text{old}} &= \text{arg} \max_k \frac{ \exp \left(-\frac{1}{2\sigma^2} \|x_i - \mu_k\|^2 \right) }{ \sum_{k=1}^K \exp \left(-\frac{1}{2\sigma^2} \|x_i - \mu_k\|^2 \right)  } \nonumber \\
&=  \text{arg} \max_k  \log \exp \left(-\frac{1}{2\sigma^2} \|x_i - \mu_k\|^2 \right)  \nonumber \\
&=  \text{arg} \min_k  \frac{1}{2\sigma^2}  \|x_i - \mu_k\|^2   \nonumber \\
&=  \text{arg} \min_k  \|x_i - \mu_k\|^2   \nonumber 
\end{align}
$$

and the means
$$
\mu_{k}^{\text{new}}  = \frac{1}{\sum_{i=1}^N \hat r_{ik}^{\text{old}}} \sum_{i=1}^N \hat r_{ik}^{\text{old}} x_i
$$


Hence the EM for K-means reduces to

1. Update "hard" responsibilities
1. Update the cluster means


## References and additional  material


- Bishop, "Pattern recognition and machine learning", **Chapter 9**
- Murphy, "Machine Learning: A Probabilistic Perspective", *MIT Press*, 2012, **Chapter 11**
- [Scikit-learn article: Setting the concentration prior](https://scikit-learn.org/stable/auto_examples/mixture/plot_concentration_prior.html)
- [Variational GMM in pymc3](https://docs.pymc.io/notebooks/gaussian-mixture-model-advi.html)


